# DHT22 Prediction Model

아래 중간 중간 `#빈칸`을 채우며 적절히 설계해보세요. 끝까지 실행하면 `temp_predict.onnx` 및 `humi_predict.onnx` 파일이 생성됩니다.

In [19]:
import json
import torch
import torch.nn as nn
import torch.onnx
import numpy as np
from torch.utils.data import DataLoader, Dataset

### Class Definitions

In [20]:
class DHTDataset(Dataset):
    def __init__(self, key):
        self.data = []
        self.target = []
        self.key = key

        db = open("db.json", "r")
        humi_temp = json.load(db)

        for i in range(len(humi_temp.get(self.key))-62):
            tmp_data = []
            tmp_data.append([float(humi_temp.get(self.key)[i][1])])
            tmp_data.append([float(humi_temp.get(self.key)[i+1][1])])
            tmp_data.append([float(humi_temp.get(self.key)[i+2][1])])
            self.data.append(torch.as_tensor(tmp_data))

            self.target.append(
                torch.as_tensor([float(humi_temp.get(self.key)[i+62][1])]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        data, target = self.data[index], self.target[index]
        return data, target

In [21]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()

        self.linear = nn.Sequential(
            nn.Linear(3, 32),  # input size = 3 (3 time steps), output size = 32
            nn.ReLU(),
            nn.Linear(32, 1)   # input size = 32, output size = 1 (predicted value)
        )

    def forward(self, x):
        x = x.view(-1, 3)
        x = self.linear(x)
        return x

### Function Definitions

##### A function to split dataset into several data loaders

In [22]:
def splitDataset(dataset):
    test_size = int(len(dataset)/6)
    val_size = int(len(dataset)/6)
    train_size = len(dataset) - test_size - val_size

    trainset, valset, testset = torch.utils.data.random_split(
        dataset, [train_size, val_size, test_size])

    trainloader = DataLoader(trainset, batch_size=16, shuffle=True)
    validloader = DataLoader(valset, batch_size=16, shuffle=True)
    testloader = DataLoader(testset, batch_size=16, shuffle=True)

    return trainloader, validloader, testloader

##### A function to train model

In [23]:
def train_model(model, patience, num_epochs, train_loader, valid_loader):
    train_losses = []
    valid_losses = []
    mean_train_losses = []
    mean_valid_losses = []
    p = 0
    min_valid_loss = float("inf")

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(1, num_epochs + 1):

        model.train()
        for train_data, train_target in train_loader:
            if torch.cuda.is_available():
                device = torch.device("cuda")
                train_data = train_data.to(device, dtype=torch.float)
                train_target = train_target.to(device, dtype=torch.float)
            optimizer.zero_grad()
            output = model(train_data)
            loss = criterion(output, train_target)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()

        for valid_data, valid_target in valid_loader:
            if torch.cuda.is_available():
                device = torch.device("cuda")
                valid_data = valid_data.to(device, dtype=torch.float)
                valid_target = valid_target.to(device, dtype=torch.float)
            output = model(valid_data)
            loss = criterion(output, valid_target)
            valid_losses.append(loss.item())

        train_loss = np.mean(train_losses)
        valid_loss = np.mean(valid_losses)
        mean_train_losses.append(train_loss)
        mean_valid_losses.append(valid_loss)

        if min_valid_loss > valid_loss:
            min_valid_loss = valid_loss
            print(f"min_valid_loss: {min_valid_loss}")

        epoch_len = len(str(num_epochs))
        print_msg = (f"[{epoch:>{epoch_len}}/{num_epochs:>{epoch_len}}] " +
                     f"train_loss: {train_loss:.5f} " +
                     f"valid_loss: {valid_loss:.5f} ")
        print(print_msg)

        train_losses = []
        valid_losses = []

        if min_valid_loss < valid_loss and epoch > 1:
            p = p + 1
            print(f'patience: {p}')
        else:
            p = 0
            torch.save(model.state_dict(), "bestmodel.pt")
            print("Saving Model...")

        if patience == p:
            print("Early stopping")
            break

    model.load_state_dict(torch.load("bestmodel.pt"))

    return model

### Training

##### Load datasets and get dataloaders

In [24]:
tempDataset = DHTDataset("temperature")
humiDataset = DHTDataset("humidity")

temp_trainloader, temp_validloader, temp_testloader = splitDataset(tempDataset)
humi_trainloader, humi_validloader, humi_testloader = splitDataset(humiDataset)

##### Make models

In [25]:
tempModel = SimpleModel()
humiModel = SimpleModel()

if torch.cuda.is_available():
    tempModel = tempModel.cuda()
    humiModel = humiModel.cuda()

##### Do training with earlystopping

In [26]:
temp_predict_model = train_model(tempModel, patience=30, num_epochs=1000,
                                 train_loader=temp_trainloader,
                                 valid_loader=temp_validloader)
torch.save(temp_predict_model.state_dict(), "temp_predict.pt")

min_valid_loss: 725.866455078125
[   1/1000] train_loss: 909.06105 valid_loss: 725.86646 
Saving Model...
min_valid_loss: 456.40069580078125
[   2/1000] train_loss: 592.00538 valid_loss: 456.40070 
Saving Model...
min_valid_loss: 262.5117492675781
[   3/1000] train_loss: 359.59650 valid_loss: 262.51175 
Saving Model...
min_valid_loss: 130.82372665405273
[   4/1000] train_loss: 195.70956 valid_loss: 130.82373 
Saving Model...
min_valid_loss: 52.98512077331543
[   5/1000] train_loss: 90.19750 valid_loss: 52.98512 
Saving Model...
min_valid_loss: 15.888832092285156
[   6/1000] train_loss: 32.77200 valid_loss: 15.88883 
Saving Model...
min_valid_loss: 3.0872669219970703
[   7/1000] train_loss: 8.51137 valid_loss: 3.08727 
Saving Model...
min_valid_loss: 0.29253584146499634
[   8/1000] train_loss: 1.34732 valid_loss: 0.29254 
Saving Model...
min_valid_loss: 0.0066064961720258
[   9/1000] train_loss: 0.08715 valid_loss: 0.00661 
Saving Model...
[  10/1000] train_loss: 0.00792 valid_loss: 0.0

In [27]:
humi_predict_model = train_model(humiModel, patience=30, num_epochs=1000,
                                 train_loader=humi_trainloader,
                                 valid_loader=humi_validloader)
torch.save(humi_predict_model.state_dict(), "humi_predict.pt")

min_valid_loss: 457.66712951660156
[   1/1000] train_loss: 524.55370 valid_loss: 457.66713 
Saving Model...
min_valid_loss: 349.69161224365234
[   2/1000] train_loss: 407.66732 valid_loss: 349.69161 
Saving Model...
min_valid_loss: 256.2993965148926
[   3/1000] train_loss: 306.40526 valid_loss: 256.29940 
Saving Model...
min_valid_loss: 178.46134185791016
[   4/1000] train_loss: 220.34139 valid_loss: 178.46134 
Saving Model...
min_valid_loss: 112.88199424743652
[   5/1000] train_loss: 147.79663 valid_loss: 112.88199 
Saving Model...
min_valid_loss: 62.34193992614746
[   6/1000] train_loss: 88.92966 valid_loss: 62.34194 
Saving Model...
min_valid_loss: 28.923904418945312
[   7/1000] train_loss: 46.09435 valid_loss: 28.92390 
Saving Model...
min_valid_loss: 10.710834503173828
[   8/1000] train_loss: 19.72993 valid_loss: 10.71083 
Saving Model...
min_valid_loss: 3.141530930995941
[   9/1000] train_loss: 6.71511 valid_loss: 3.14153 
Saving Model...
min_valid_loss: 0.9409873634576797
[  10/

### ONNX Coversion

In [28]:
def convert_to_onnx(saved_file_name):
    saved_model = SimpleModel()
    saved_model.load_state_dict(torch.load(saved_file_name))
    saved_model.eval()
    torch.onnx.export(
        saved_model,
        torch.randn((1, 3, 1)),
        saved_file_name[:-3] + '.onnx',
        opset_version=11,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"]
    )

In [29]:
convert_to_onnx("temp_predict.pt")
convert_to_onnx("humi_predict.pt")

In [30]:
# End of Document.

In [31]:
humi_predict_model

SimpleModel(
  (linear): Sequential(
    (0): Linear(in_features=3, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=1, bias=True)
  )
)